In [ ]:
%pip install pandas duckdb matplotlib seaborn

import pandas as pd
import duckdb
import os
import matplotlib.pyplot as plt
import re
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
speed_file = '../spec_speed/spec-median-cint2006cint2017-tpc-cpus.csv'
# if you want to recreate tpc_xeon
# file_path = '../tpc_xeon.csv'
if os.path.exists(speed_file):
    df_speed = pd.read_csv(speed_file, sep=",")
else:
    raise FileNotFoundError(f"The file {speed_file} does not exist.")

rate_file = '../spec_rate/spec-median-rint2006rint2017-tpc-cpus.csv'
if os.path.exists(rate_file):
    df_rate = pd.read_csv(rate_file, sep=",")
else:
    raise FileNotFoundError(f"The file {rate_file} does not exist.")

key_year = 'model_launch_year'
key_speed = "median_spec_int_speed_perf"
key_speed_multi = "perf_times_cores"
key_speed_tdp = "speed_per_tdp"
key_speed_multi_tdp = "speed_multi_per_tdp"
key_tdp = "TDP/CPU (W)"
key_cores_per_cpu = "cores_per_cpu"
key_cores = "cores"
key_processors = "processors"

key_rate = "median_spec_int_rate_perf"
key_rate_tdp = "rate_per_tdp"


df_speed[key_speed_tdp] = df_speed[key_speed] / df_speed[key_tdp]
df_rate[key_rate_tdp] = df_rate[key_rate] / df_rate[key_tdp]

# display(df)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({'text.usetex': True
                            , 'pgf.rcfonts': False
                            , 'text.latex.preamble': r"""\usepackage{iftex}
                                            \ifxetex
                                                \usepackage[libertine]{newtxmath}
                                                \usepackage[tt=false]{libertine}
                                                \setmonofont[StylisticSet=3]{inconsolata}
                                            \else
                                                \RequirePackage[tt=false, type1=true]{libertine}
                                            \fi"""
                         })


colors_qualitative = ['#abd9e9', '#fdae61', '#d7191c']

# Assuming df is your DataFrame and key_year, key_perf, key_spec_perf, key_perf_per_tdp are your column names
df_speed = df_speed.sort_values(by=key_year)
df_rate = df_rate.sort_values(by=key_year)


df_speed_first_year = df_speed[df_speed[key_year] == df_speed[key_year].iloc[0]]
print(df_speed_first_year)
df_speed["speedup"] = (df_speed[key_speed] / df_speed_first_year[key_speed].mean())

df_rate_first_year = df_rate[df_rate[key_year] == df_rate[key_year].iloc[0]]
print(df_rate_first_year)
df_rate["speedup"] = (df_rate[key_rate] / df_rate_first_year[key_rate].mean())
df_rate["relative_rate_tdp"] = df_rate[key_rate_tdp] / df_rate_first_year[key_rate_tdp].mean()

# display(df_speed)
display(df_rate)

# Define the figures size and x-axis rotation
figsize = (9, 2)
x_rotation = 0
window_size = 3  # Adjust the window size as needed

# Create a single figure with three subplots
fig, axes = plt.subplots(1, 3, figsize=figsize, sharex=True, sharey=False)

years = list(range(2006, 2024))
years_only_every_n = [year if i % 3 == 0 else '' for i, year in enumerate(years)]

# First subplot (originally second)
x_val = key_year
# y_val = key_speed
y_val = "speedup"

agg_df = df_speed.groupby(key_year)[y_val].mean().reset_index()
moving_agg_df = agg_df.rolling(window=window_size, center=True).mean()

axes[0].scatter(df_speed[x_val], df_speed[y_val], s=len(df_speed[x_val]) * 0.25, label='CPUs', color=colors_qualitative[1])
axes[0].plot(moving_agg_df[x_val], moving_agg_df[y_val], color=colors_qualitative[2], linestyle='--', label=f'{window_size}-Year Moving Average')
axes[0].set_xlabel('CPU Launch Year')
axes[0].set_ylabel("SPECspeed\nSpeedup Factor")
axes[0].set_xticks(years)
axes[0].set_xticklabels(years_only_every_n)
axes[0].yaxis.set_major_locator(ticker.MultipleLocator(1))
axes[0].tick_params(axis='x', rotation=x_rotation)
axes[0].set_ylim(bottom=0)
# axes[0].set_yscale('symlog')
axes[0].set_title("")

# Second subplot (originally first)
# y_val = key_rate
y_val = "speedup"

agg_df = df_rate.groupby(key_year)[y_val].mean().reset_index()
moving_agg_df = agg_df.rolling(window=window_size, center=True).mean()

axes[1].scatter(df_rate[x_val], df_rate[y_val], s=len(df_rate[x_val]) * 0.25, label='CPUs', color=colors_qualitative[1])
axes[1].plot(moving_agg_df[x_val], moving_agg_df[y_val], color=colors_qualitative[2], linestyle='--', label=f'{window_size}-Year Moving Average')
axes[1].set_xlabel('CPU Launch Year')
axes[1].set_ylabel("SPECrate\nSpeedup Factor")
axes[1].set_xticks(years)
axes[1].set_xticklabels(years_only_every_n)
axes[1].yaxis.set_major_locator(ticker.MultipleLocator(5))
axes[1].tick_params(axis='x', rotation=x_rotation)
axes[1].set_ylim(bottom=0)
# axes[1].set_yscale('symlog')
axes[1].set_title("")

# Get the y-axis limits of the second subplot (axes[1])
# ylim = axes[1].get_ylim()

# Apply the same y-axis limits to the first subplot (axes[0])
# axes[0].set_ylim(ylim)

# Third subplot
# y_val = key_rate_tdp
y_val = "relative_rate_tdp"

agg_df = df_rate.groupby(key_year)[y_val].mean().reset_index()
moving_agg_df = agg_df.rolling(window=window_size, center=True).mean()

axes[2].scatter(df_rate[x_val], df_rate[y_val], s=len(df_rate[x_val]) * 0.25, label='CPUs', color=colors_qualitative[1])
axes[2].plot(moving_agg_df[x_val], moving_agg_df[y_val], color=colors_qualitative[2], linestyle='--', label=f'{window_size}-Year Moving Average')
axes[2].set_xlabel('CPU Launch Year')
axes[2].set_ylabel("SPECrate per TDP\nImprovement Factor")
# axes[2].set_yscale('log')
axes[2].set_xticks(years)
axes[2].set_xticklabels(years_only_every_n)
axes[2].yaxis.set_major_locator(ticker.MultipleLocator(5))
axes[2].tick_params(axis='x', rotation=x_rotation)
# axes[2].set_ylim(bottom=0)
axes[2].set_title("")

# Create a single legend for all subplots
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3, handletextpad=0.3, handlelength=1.0, markerscale=0.7, labelspacing=0.1)

# Adjust layout to make room for the legend and captions
plt.tight_layout()
plt.subplots_adjust(top=0.85, bottom=0.25, left=0.05, right=0.95)

# Add captions below each subplot
fig.text(0.16, -0.15, '(a) Single-thread performance', ha='center', fontsize=10)
fig.text(0.50, -0.15, '(b) Multi-thread performance', ha='center', fontsize=10)
fig.text(0.84, -0.15, '(c) Energy efficiency', ha='center', fontsize=10)

# Save and show the plot
plt.savefig("../figures/tcp_combined_perf.pdf", bbox_inches='tight')
plt.savefig("../figures/tcp_combined_perf.svg", bbox_inches='tight')
plt.show()
plt.close()
